In [15]:
from preprocessing.prep import prepare_paths, prepare_labels, remover_sublistas_redundantes

In [24]:
from dataset.labels import get_all_structure, get_labels_name

In [2]:
import pandas as pd

In [3]:
import os
import json
from tqdm import tqdm

In [4]:

args = pd.Series({
    "root_dir": "/mnt/disks/data/",
    "dataset_path": "/mnt/disks/data/fma/fma_large",
    "embeddings": "music_style",
    "sequence_size": 1280,
    "train_id": "hierarchical_hiclass",
    'sample_size': 1/100
})



In [48]:
args, tracks_df = prepare_paths(args)


In [49]:
tracks_df

,track_genres_all,track_id_,file_path
55941,"[12, 15, 89, 314, 25]",84597,/mnt/disks/data/fma/fma_large/084/084597.mp3
75291,"[250, 38]",115882,/mnt/disks/data/fma/fma_large/115/115882.mp3
4467,"[42, 15]",8997,/mnt/disks/data/fma/fma_large/008/008997.mp3
100747,"[25, 12]",149728,/mnt/disks/data/fma/fma_large/149/149728.mp3
57196,"[4, 5, 38, 74, 250, 187]",86111,/mnt/disks/data/fma/fma_large/086/086111.mp3
...,...,...,...
30799,"[25, 12, 85]",47743,/mnt/disks/data/fma/fma_large/047/047743.mp3
46100,"[21, 15]",69230,/mnt/disks/data/fma/fma_large/069/069230.mp3
94237,"[17, 27, 12]",140887,/mnt/disks/data/fma/fma_large/140/140887.mp3
18585,"[25, 12, 111]",30132,/mnt/disks/data/fma/fma_large/030/030132.mp3


In [55]:
def prepare_labels(tracks_df,args):
    ##### Labels
    # Loand genres df
    genres_df = pd.read_csv(os.path.join(args.metadata_path, 'genres.csv'))
    # Mapear os identificadores numéricos de gêneros para os nomes dos gêneros
    genre_names = dict(zip(genres_df['genre_id'], genres_df['title']))

    # Inicialize uma lista para armazenar todos os caminhos de gêneros para cada exemplo
    estruturas = []

    # Iterar sobre as faixas e seus gêneros associados
    for track_genres in tracks_df['track_genres_all']:
        #caminho_name = [genre_names[genre_id] for genre_id in track_genres]
        caminho_id = [get_all_structure(genre_id, genres_df) for genre_id in track_genres]
        estruturas.append(caminho_id)

    for idx, caminho in enumerate(estruturas):
        caminho.sort(key=len, reverse=True)
        estruturas[idx] = remover_sublistas_redundantes(caminho)

    ## Get structure form hierarchical classification
    tracks_df['full_genre_id'] = estruturas
    tracks_df = tracks_df[['track_id_', 'full_genre_id']]

    ## Calculate labels_size
    max_depth = tracks_df.full_genre_id.apply(lambda x: len(x))
    max_depth = int(max_depth.max())
    
    labels_name = []
    for level in range(max_depth):
        labels_name.append(f'label_{level+1}')
    tqdm.pandas()
    ## Gnetare categories_df
    #labels =  tracks_df.full_genre_id.progress_apply(lambda x: get_labels_per_level(x))

    #all_levels = categories_df.label5.progress_apply(lambda x: split_label(x))
    all_labels = []
    for idx, row in enumerate(tracks_df.full_genre_id):
        for labels in row:
            labels.extend([0] * (max_depth - len(labels)))
            all_labels.append(labels)
            
    categories_df = pd.DataFrame(all_labels, columns=labels_name).drop_duplicates()
    
    categories_df[f'label_{max_depth}_name'] = [get_labels_name(categorie, genres_df) for categorie in categories_df.values]

    print(categories_df)

    data = __create_labels__(categories_df, max_depth)
    # Write labels file
    with open(args.categories_labels_path, 'w+') as f:
        f.write(json.dumps(data))

    return tracks_df, data

In [61]:
def __create_labels__(categories_df, max_depth=3):
    data = {}
    for level in range(max_depth):
        level+=1
        level_name = f'label_{level}'
        idx = 0
        data[level_name] = {}
        data[f'{level_name}_name'] = {}
        data[f'{level_name}_inverse'] = []
        data[f'{level_name}_count'] = 0
        slice_df = categories_df[[level_name, f'label_{max_depth}_name']].drop_duplicates()
        
        for id_x, row in enumerate(slice_df.values.tolist()):
            cat, name = row
            data[level_name][cat] = idx
            data[f'{level_name}_name'][cat] = '>'.join(name.split('>')[:level])
            data[f'{level_name}_inverse'].append(cat)
            data[f'{level_name}_count'] = idx + 1
    

    for values in categories_df.values:
        label_name = values[-1]
        labels = values[:-1]
        for level, label in zip(range(max_depth),labels):
            data[f'label_{level+1}_name'][int(label)] = '>'.join(label_name.split('>')[:level+1])
            
        
    #     name1 = '>'.join(name5.split('>')[:1])
    #     name2 = '>'.join(name5.split('>')[:2])
    #     name3 = '>'.join(name5.split('>')[:3])
    #     name4 = '>'.join(name5.split('>')[:4])

    #     data['label1_name'][cat1] = name1
    #     data['label2_name'][cat2] = name2
    #     data['label3_name'][cat3] = name3
    #     data['label4_name'][cat4] = name4
    #     data['label5_name'][cat5] = name5

    return data


In [62]:
tracks_df_new, data = prepare_labels(tracks_df,args)

      label_1  label_2  label_3  label_4  label_5  \
0          12       25       89        0        0   
1          15        0        0        0        0   
2          38      250        0        0        0   
3          15       42        0        0        0   
4          12       25        0        0        0   
...       ...      ...      ...      ...      ...   
1365        2       86        0        0        0   
1556       20      378        0        0        0   
1669       20       65      189        0        0   
1697       12       25       71        0        0   
1738       20       65      166        0        0   

                                           label_5_name  
0               Rock>Punk>Post-Punk>Post-Punk>Post-Punk  
1     Electronic>Electronic>Electronic>Electronic>El...  
2              Experimental>Improv>Improv>Improv>Improv  
3     Electronic>Ambient Electronic>Ambient Electron...  
4                              Rock>Punk>Punk>Punk>Punk  
...            

In [68]:
data['label_1_name']

{12: 'Rock',
 15: 'Electronic',
 38: 'Experimental',
 4: 'Jazz',
 5: 'Classical',
 1235: 'Instrumental',
 10: 'Pop',
 17: 'Folk',
 21: 'Hip-Hop',
 20: 'Spoken',
 2: 'International',
 9: 'Country',
 3: 'Blues',
 13: 'Easy Listening',
 14: 'Soul-RnB',
 8: 'Old-Time / Historic'}

In [ ]:
genres_df = pd.read_csv(os.path.join(args.metadata_path, 'genres.csv'))

In [16]:
genres_df[genres_df['parent'] == 0]

,genre_id,#tracks,parent,title,top_level
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
7,8,868,0,Old-Time / Historic,8
8,9,1987,0,Country,9
9,10,13845,0,Pop,10
11,12,32923,0,Rock,12
12,13,730,0,Easy Listening,13
13,14,1499,0,Soul-RnB,14


In [ ]:
#split_dataset(tracks_df,args)